In [99]:
from datetime import datetime
import re
import rpyc
from rpyc.utils.classic import teleport_function

conn = rpyc.classic.connect("192.168.4.67")

def get_disks():
    from os import listdir
    import re
    disks = sorted([x for x in listdir('/dev/') if 
               re.findall('sd[a-z]+$',str(x))])
    return disks


def run_command(command):
    import subprocess
    import shlex

    lst = []
    counter = 0
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            return lst

        if output:
            lst.append(output.strip())
            counter += 1

    rc = process.poll()
    return rc


rc = teleport_function(conn,run_command)

start = datetime.now()
for disk in disks():
    print disk
    smart_data = rc('sudo smartctl -a /dev/{}'.format(disk))
    for line in smart_data:
        match = re.findall('serial.*',line,re.IGNORECASE)
        if match:
            print match
            break
finish = datetime.now()
print finish-start

sda
['Serial number:        6SD3FDPD0000B145K2LQ']
sdb
['Serial number:        6SD3FKFY0000B145J6JE']
sdc
['Serial number:        PQGHH1NB']
sdd
['Serial number:        S421JCDK0000M6455KWA']
sde
['Serial number:        D301PC40RLGG1216']
sdf
['Serial number:        6XM25PNF']
sdg
['Serial number:        3NM6EFR4']
sdh
['Serial number:        PCXGXKME']
sdi
['Serial number:        D301PC40RKS21216']
sdj
['Serial number:        PCXNND9E']
sdk
['Serial number:        6SD3FBLC0000B144PM86']
sdl
['Serial number:        PCXLWV0E']
sdm
['Serial number:        PQGHV7DB']
sdn
['Serial number:        D0A1P9A058R70944']
sdo
0:00:14.124679


In [ ]:
time ls /dev/|grep sd |grep -v [0-9]|sort|xargs -I {} sudo smartctl -a /dev/{} |grep -i serial

Serial number:        6SD3FDPD0000B145K2LQ
Serial number:        6SD3FKFY0000B145J6JE
Serial number:        PQGHH1NB
Serial number:        S421JCDK0000M6455KWA
Serial number:        D301PC40RLGG1216
Serial number:        6XM25PNF
Serial number:        3NM6EFR4
Serial number:        PCXGXKME
Serial number:        D301PC40RKS21216
Serial number:        PCXNND9E
Serial number:        6SD3FBLC0000B144PM86
Serial number:        PCXLWV0E
Serial number:        PQGHV7DB
Serial number:        D0A1P9A058R70944

real	0m11.860s
user	0m0.500s
sys	0m0.104s


In [ ]:
time parallel sudo smartctl -a /dev/{} < <(ls /dev|grep sd|grep -v [0-9])|grep -i serial
Serial number:        PCXNND9E
Serial number:        PCXGXKME
Serial number:        PCXLWV0E
Serial number:        D0A1P9A058R70944
Serial number:        D301PC40RKS21216
Serial number:        PQGHH1NB
Serial number:        PQGHV7DB
Serial number:        S421JCDK0000M6455KWA
Serial number:        D301PC40RLGG1216
Serial number:        6SD3FDPD0000B145K2LQ
Serial number:        6SD3FKFY0000B145J6JE
Serial number:        6SD3FBLC0000B144PM86
Serial number:        3NM6EFR4
Serial number:        6XM25PNF

real	0m2.803s
user	0m0.600s
sys	0m0.172s
